In [1]:
import requests
from bs4 import BeautifulSoup

In [6]:
import os
from io import StringIO
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager, process_pdf
import logging

In [ ]:
def download(url, filename):
    if os.path.exists(filename) is False:
        with open(filename,"wb") as file:
            res = requests.get(url)
            file.write(res.content)

In [3]:
def read_pdf_file(pdfFile):
    logging.propagate = False 
    logging.getLogger().setLevel(logging.ERROR)
    with open(pdfFile, mode='rb') as file:
        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr, laparams=laparams)
        process_pdf(rsrcmgr, device, file)
        device.close()

        content = retstr.getvalue()
        retstr.close()
    
    return content

In [ ]:
def sec1(page):
    res = requests.get('http://www.kats.go.kr/content.do?cmsid=304&page=' + str(page)).text
    soup = BeautifulSoup(res,'html.parser')
    crawl_ls = soup.select('table[summary="제품안전 전기용품리스트로 , 기준번호 , 기준명 , 고시일 , 조회수 , 첨부파일  제공"] tbody tr')
    
    return crawl_ls

In [ ]:
def getContent_sec1(crawl_ls, fp):
    for crawl in crawl_ls:
        file_content = ''
        std_num = crawl.select('td')[0].text
        std_name = crawl.select('td')[1].text
        date = crawl.select('td')[2].text
        
        # 첨부파일 1개라고 가정
        crawl_file = crawl.select('td[class="no_b_right"] a')[0]
        file_name = crawl_file.select('img')[0].get('title')
        file_path = 'http://www.kats.go.kr' + crawl_file.get('href')

        if 'pdf' in file_name:
            download(file_path, file_name)
            file_content = read_pdf_file(file_name).replace('\n','')

        # 파일에 저장 TODO: ','무시하기
        if file_content is not '':
            fp.write('"{0}","{1}","{2}","{3}","{4}"\n'.format(std_num, std_name, date, file_name, file_content.strip()))
            
    return fp

In [ ]:
fp = open('sec1.csv', 'wt', encoding = 'utf-8')
for page in range(1,2):
    url_ls = sec1(page)
    fp = getContent_sec1(url_ls, fp)
fp.close()

In [ ]:
def sec2(page):
    url_ls = []
    res = requests.get('http://www.kats.go.kr/content.do?cmsid=527&page='+str(page)).text
    soup = BeautifulSoup(res,'html.parser')
    crawl_ls = soup.select('table[summary="생활용품 안전기준리스트로 , 구분 , 분야 , 제목 , 작성자 , 작성일 , 조회수  제공"] tbody tr .tbleft a')
    for crawl in crawl_ls:
        url_ls.append(crawl.get('href'))
    return url_ls

In [ ]:
def sec3(page):
    url_ls = []
    res = requests.get('http://www.kats.go.kr/content.do?cmsid=530&page='+str(page)).text
    soup = BeautifulSoup(res,'html.parser')
    crawl_ls = soup.select('table[summary="어린이제품 안전기준리스트로 , 구분 , 제목 , 작성자 , 작성일 , 조회수  제공"] tbody tr .tbleft a')
    for crawl in crawl_ls:
        url_ls.append(crawl.get('href'))
    return url_ls

In [ ]:
# sec2, sec3는 기준번호, 고시일이 없음(우선 구분, 분야, 작성일로 대체)
def getContent(url_ls, fp):
    for url in url_ls:
        file_content = ''
        res = requests.get('http://www.kats.go.kr' + url).text
        soup = BeautifulSoup(res,'html.parser')
        
        crawl = soup.select('table')[0]
        std_name = crawl.select('thead th[class="no_b_right tbleft"]')[0].text
        date = crawl.select('tbody td[class="no_b_right"]')[0].text
        crawl_body = crawl.select('tbody td[class="no_b_right tbleft"]')
        div = crawl_body[0].text
        field = crawl_body[1].text
        
        for crawl_file in crawl_body[2].select('a'):
            file_name = crawl_file.text
            if 'pdf' in file_name:
                file_path = 'http://www.kats.go.kr' + crawl_file.get('href')
                download(file_path, file_name)
                file_content = read_pdf_file(file_name).replace('\n','')
            elif len(crawl_file) == 1 and 'hwp' in file_name:
                pass
#                 download(file_path, file_name) # hwp 파일만 있는 경우
        
        if file_content is not '':
            fp.write('"{0}","{1}","{2}","{3}","{4}","{5}"\n'.format(div, field, std_name, date, file_name, file_content.strip()))

    return fp

In [ ]:
fp = open('sec2.csv', 'wt', encoding = 'utf-8')
for page in range(1,2):
    url_ls = sec2(page)
    fp = getContent(url_ls, fp)
fp.close()

In [ ]:
fp = open('sec3.csv', 'wt', encoding = 'utf-8')
for page in range(1,2):
    url_ls = sec3(page)
    fp = getContent(url_ls, fp)
fp.close()